In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from scipy import stats
import surprise

In [2]:
########## Read files ##########

# Load the movielens-100k dataset (download it if needed),
# and split it into 3 folds for cross-validation.
data = surprise.Dataset.load_builtin('ml-100k')

# userId, movieId, rating, timestamp
ratings = pd.read_csv(data.ratings_file, sep='\t',names=['userId', 'movieId', 'rating', 'timestamp'])

In [3]:
########## Spliting into train and test ##########
train_ratings, test_ratings = model_selection.train_test_split(ratings, test_size = 0.33)

In [4]:
########## Investigating the data and making auxilary data structures ##########

trainUserIds = train_ratings.userId.unique()
trainUserIds.sort()
print ("%20s" % "Train user IDs: " + str((trainUserIds.shape, trainUserIds)))

testUserIds = test_ratings.userId.unique()
testUserIds.sort()
print ("%20s" % "Test user IDs: " + str((testUserIds.shape, testUserIds)))

trainMovieIds = train_ratings.movieId.unique()
trainMovieIds.sort()
print ("%20s" % "Train movie IDs: " + str((trainMovieIds.shape, trainMovieIds)))

testMovieIds = test_ratings.movieId.unique()
testMovieIds.sort()
print ("%20s" % "Test movie IDs: " + str((testMovieIds.shape, testMovieIds)))

userIds = ratings.userId.unique()
userIds.sort()
print ("%20s" % "User IDs: " + str((userIds.shape, userIds)))

movieIds = ratings.movieId.unique()
movieIds.sort()
print ("%20s" % "Movie IDs: " + str((movieIds.shape, movieIds)))

# We can use the UserId as an index of the matrix, but we cannot use movieId
# Let's make index to ID dictionary anyway for both for robustness

userCount = userIds.size
movieCount = movieIds.size

userIndexToIdDict = dict()
userIdToIndexDict = dict()
for i in range(userCount):
    userIndexToIdDict[i] = userIds[i]
    userIdToIndexDict[userIds[i]] = i

movieIndexToIdDict = dict()
movieIdToIndexDict = dict()
for i in range(movieCount):
    movieIndexToIdDict[i] = movieIds[i]
    movieIdToIndexDict[movieIds[i]] = i

    Train user IDs: ((943,), array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175

In [5]:
########## Constructing user/movie rating matrices ##########

# Constructing a train matrix
trainMatrix = np.zeros((userCount, movieCount))
for rating in train_ratings.itertuples():
    trainMatrix[int(userIdToIndexDict[rating.userId]), int(movieIdToIndexDict[rating.movieId])] = rating.rating
    
# Constructing a test matrix
testMatrix = np.zeros((userCount, movieCount))
for rating in test_ratings.itertuples():
    testMatrix[int(userIdToIndexDict[rating.userId]), int(movieIdToIndexDict[rating.movieId])] = rating.rating

In [6]:
########## Calculating simmilarity matrices ##########

appliedMetrics = np.array(['cosine', 'euclidean'])
appliedMetricCount = appliedMetrics.shape[0]
userSimilarities = dict()

# cosine simmilarity:
## Here is an explanation why cosine distance was returning 0 instead of 1 (discussed on project presentation)
## Long story short it is a DISTANCE, not similarity (snippet from documentation):
##
## Signature: metrics.pairwise.cosine_distances(X, Y=None)
## Docstring:
## Compute cosine distance between samples in X and Y.
##
## Cosine distance is defined as 1.0 minus the cosine similarity.
userSimilarities[appliedMetrics[0]] = 1 - metrics.pairwise_distances(trainMatrix, metric = appliedMetrics[0])

# euclidean simmilarity
userSimilaritiyEuclidean = metrics.pairwise_distances(trainMatrix, metric = appliedMetrics[1])
userSimilarities[appliedMetrics[1]] = 1 - userSimilaritiyEuclidean / userSimilaritiyEuclidean.max()

In [7]:
########## Calculating the predictions ##########

# First algorithm, taken as a basic algorithm is from andjelkaz' website
def prediction_basic(matrix, user_similarity):
    x_mean = matrix.mean(axis = 1)
    matrix_diff = (matrix - x_mean[:, np.newaxis])
    p = x_mean[:, np.newaxis] + user_similarity.dot(matrix_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
    return p

def prediction_basic_with_threshold(matrix, user_similarity, threshmin):
    x_mean = matrix.mean(axis = 1)
    user_similarity = stats.threshold(user_similarity, threshmin=threshmin, threshmax=1, newval=0)
    matrix_diff = (matrix - x_mean[:, np.newaxis])
    p = x_mean[:, np.newaxis] + user_similarity.dot(matrix_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
    return p

def prediction_basic_no_mean(matrix, user_similarity):
    p = user_similarity.dot(matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T
    return p

def prediction_basic_no_mean_with_threshold(matrix, user_similarity, threshmin):
    user_similarity = stats.threshold(user_similarity, threshmin=threshmin, threshmax=1, newval=0)
    p = user_similarity.dot(matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T
    return p

def prediction_basic_k_nearest_neighbours(matrix2, user_similarity2, k):
    userSimilarity = np.matrix.copy(user_similarity2)
    for i in range(userSimilarity.shape[1]):
        userSimilarityForUser = userSimilarity[i]
        kNearestNeighbours = userSimilarityForUser.argsort()[-(k + 1):][::-1][1:]

        #print ("Count before: " + str(len(userSimilarityForUser.nonzero()[0])))
        userSimilarityForUserNew = np.zeros(userSimilarity.shape[0])
        userSimilarityForUserNew[kNearestNeighbours] = userSimilarityForUser[kNearestNeighbours]
        userSimilarity[i] = userSimilarityForUserNew
        #print ("Count after (expected " + str(k) + "): " + str(len(userSimilarityForUser.nonzero()[0])))
        
    p = userSimilarity.dot(matrix2) / np.array([np.abs(userSimilarity).sum(axis=1)]).T
    p[np.bitwise_not(np.isfinite(p))] = 0
    return p


userMoviePredictions = dict()

for appliedMetric in appliedMetrics:
    userSimilarity = userSimilarities[appliedMetric]
    
    if (appliedMetric == 'cosine'):
        threshmin = 0.5
    else:
        threshmin = 0.8

    userMoviePredictions[(appliedMetric, 'basic')] = prediction_basic(testMatrix, userSimilarity)
    userMoviePredictions[(appliedMetric, 'basicWithThreshold')] = prediction_basic_with_threshold(testMatrix, userSimilarity, threshmin)
    userMoviePredictions[(appliedMetric, 'basicNoMean')] = prediction_basic_no_mean(testMatrix, userSimilarity)
    userMoviePredictions[(appliedMetric, 'basicNoMeanWithThreshold')] = prediction_basic_no_mean_with_threshold(testMatrix, userSimilarity, threshmin)
    userMoviePredictions[(appliedMetric, 'basicKNearestNeighbours')] = prediction_basic_k_nearest_neighbours(testMatrix, userSimilarity, 20)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `threshold` is deprecated!
stats.threshold is deprecated in scipy 0.17.0
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: `threshold` is deprecated!
stats.threshold is deprecated in scipy 0.17.0


In [8]:
########## Scoring different models ##########

def evaluation(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return np.sqrt(metrics.mean_squared_error(ground_truth, prediction))

evaluations = dict()

for userMoviePrediction in userMoviePredictions:
    evaluations[userMoviePrediction] = evaluation(userMoviePredictions[userMoviePrediction], testMatrix)

evaluations

{('cosine', 'basic'): 3.328493579223621,
 ('cosine', 'basicKNearestNeighbours'): 3.1510527897000511,
 ('cosine', 'basicNoMean'): 3.3684019774865583,
 ('cosine', 'basicNoMeanWithThreshold'): 0.26054661112265409,
 ('cosine', 'basicWithThreshold'): 0.26035221607613801,
 ('euclidean', 'basic'): 3.4276165604054247,
 ('euclidean', 'basicKNearestNeighbours'): 3.5441196565851705,
 ('euclidean', 'basicNoMean'): 3.4967470015463356,
 ('euclidean', 'basicNoMeanWithThreshold'): 0.69439196345629572,
 ('euclidean', 'basicWithThreshold'): 0.69345757934403107}

In [9]:
########## Scoring models from library 'surprise' ##########

data.split(n_folds=3)

# Evaluate performances of different algorithms.
perfKNNBasic = surprise.evaluate(surprise.KNNBasic(), data, measures=['RMSE'])
perfKNNWithMeans = surprise.evaluate(surprise.KNNWithMeans(), data, measures=['RMSE'])
perfSVD = surprise.evaluate(surprise.SVD(), data, measures=['RMSE'])


Evaluating RMSE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9880
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9891
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9908
------------
------------
Mean RMSE: 0.9893
------------
------------
Evaluating RMSE of algorithm KNNWithMeans.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9594
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9574
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9545
------------
------------
Mean RMSE: 0.9571
------------
------------
Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9424
------------
Fold 2
RMSE: 0.9463
------------
Fold 3
RMSE: 0.9421
------

In [10]:
surprise.print_perf(perfKNNBasic)

        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.9880  0.9891  0.9908  0.9893  


In [11]:
surprise.print_perf(perfKNNWithMeans)

        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.9594  0.9574  0.9545  0.9571  


In [12]:
surprise.print_perf(perfSVD)

        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.9424  0.9463  0.9421  0.9436  
